# Leeds property price analytics

In [47]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# force full-width display if viewed in Chrome browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Download UK Land Registry price-paid data and create bigquery dataset

In [ ]:
# download data to the VM from source: https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads
!wget http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv

In [ ]:
# move to gcp bucket
!gsutil mv pp-complete.csv gs://housingasodhfmq349p78vp57pasvpfphio/pp-complete.csv

In [ ]:
# make bigquery database
!bq mk housing

In [ ]:
# create table called 'pricepaid' in housing database
!bq load --source_format=CSV \
    housing.pricepaid gs://housingasodhfmq349p78vp57pasvpfphio/pp-complete.csv \
    trans_id:STRING,price:INTEGER,date:STRING,postcode:STRING,type:STRING,newbuild:STRING,tenure:STRING,paon:STRING,saon:STRING,street:STRING,locality:STRING,town_city:STRING,district:STRING,county:STRING,ppd_type:STRING,record_status:STRING

## Query data

In [161]:
%%bigquery df

SELECT * FROM housing.pricepaid
WHERE town_city = 'LEEDS'

Downloading: 100%|██████████| 302956/302956 [00:02<00:00, 113549.75rows/s]


## Inspect dataframe

In [162]:
df.head(4)

,trans_id,price,date,postcode,type,newbuild,tenure,paon,saon,street,locality,town_city,district,county,ppd_type,record_status
0,{80B98B64-F7A8-4F75-8D6B-CF92348EA779},187000,2004-01-30 00:00,LS17 7SH,D,N,F,17,,SUNNINGDALE DRIVE,LEEDS,LEEDS,LEEDS,WEST YORKSHIRE,A,A
1,{21E5FEB5-D94C-2439-E050-A8C06205342E},267000,2014-08-26 00:00,LS7 4LY,D,N,F,1C,,CHURCH LANE,CHAPEL ALLERTON,LEEDS,LEEDS,WEST YORKSHIRE,B,A
2,{DFCBEA35-78ED-4CB1-B892-119778FCEEA5},304000,2015-09-17 00:00,LS17 8XR,D,N,F,19,,OAKDENE WAY,,LEEDS,LEEDS,WEST YORKSHIRE,A,A
3,{5BD5D826-4B53-4D3E-852A-8DA733041E3C},66000,1998-10-02 00:00,LS12 5LX,D,N,F,2,,WATER LANE,LEEDS,LEEDS,LEEDS,WEST YORKSHIRE,A,A


In [163]:
df.nunique()

trans_id         302956
price             12063
date               7702
postcode          14067
type                  5
newbuild              2
tenure                3
paon               6205
saon               2348
street             8956
locality            120
town_city             1
district             11
county                4
ppd_type              2
record_status         1
dtype: int64

In [164]:
# check data types
df.dtypes

trans_id         object
price             int64
date             object
postcode         object
type             object
newbuild         object
tenure           object
paon             object
saon             object
street           object
locality         object
town_city        object
district         object
county           object
ppd_type         object
record_status    object
dtype: object

## Pre-processing

#### 1. Postcode

In [165]:
# Some rows have missing postcodes
len(df[df['postcode'] == ''])

202

In [166]:
# Check for null entries in postcode column
len(df[df['postcode'].isnull()])

0

In [167]:
# Discard rows with empty string postcodes 
df = df[df['postcode'] != '']

In [168]:
# Check that all postcodes have space
len(df[df['postcode'].str.contains(' ')]) == len(df)

True

In [169]:
# Break down postcodes into components: area, district, sector ,and unit.
def get_postcode_component(postcode, component):
    
    """
    Returns part of all of a postcode string resolved into area, district, sector or unit. 
    
    UK postcodes take any of the following formats, where A denotes alphabetical
    and N denotes numeric:
    
    AN NAA 
    ANN NAA
    AAN NAA
    AANN NAA
    ANA NAA
    AANA NAA
    
    (source: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/611951/Appendix_C_ILR_2017_to_2018_v1_Published_28April17.pdf)
    
    - Area is the leftmost substring of alpha characters until (and excluding) a numeric character .Format: A or AA.
    - District is the substring commencing with numeric after the area, and before the space. Format: N, NN, or NA.
    - Sector is the first numeric character after the space. Format: N.
    - Unit is the rightmost substring. Format: AA.
   
    e.g.:
    >> get_postcode_component('RG40 2WD', component='sector')
    >> RG40 2   
    
    e.g. 2:
    >> get_postcode_component('EC1A 3HX', component='area')
    >> EC
    
    e.g. 3:
    >> get_postcode_component('EC1A 3HX', component='district')
    >> EC1A
    """
    
    # e.g. EC1A 3HX   /   RG40 3HX
    area_and_district = list(postcode.split(' ')[0])  # ['E', 'C', '1', 'A']   /   ['R', 'G', '4', '0']
    sector_and_unit = list(postcode.split(' ')[1])    # ['3', 'H', 'X']        /   ['3', 'H', 'X']
    
    if component == 'area':
        for idx, el in enumerate(area_and_district):
            if el.isnumeric():
                area = ''.join(area_and_district[:idx]) # EC  /  RG
                return area        

    elif component == 'district':
        for idx, el in enumerate(area_and_district):
            if el.isnumeric():
                district = ''.join(area_and_district[idx:]) # 1A  /  40
                return district
                
    elif component == 'sector':
        for idx, el in enumerate(sector_and_unit):
            if not el.isnumeric():
                sector = ''.join(sector_and_unit[:idx])   # 3   /   2
                return sector
                
    elif component == 'unit':
        for idx, el in enumerate(sector_and_unit):
            if not el.isnumeric():
                unit = ''.join(sector_and_unit[idx:])   # HX  /   QS
                return unit
    
    else:
        return None


In [170]:
# unit tests
test_postcodes = ['A1 2BC', 'A12 3BC', 'AB1 2BC', 'AB12 3CD', 'A1B 2BC', 'AB1C 2DE']

for postcode in test_postcodes:
    print(postcode, 'area: :', get_postcode_component(postcode, component='area'))
    print(postcode, 'district: :', get_postcode_component(postcode, component='district'))
    print(postcode, 'sector: :', get_postcode_component(postcode, component='sector'))
    print(postcode, 'unit: :', get_postcode_component(postcode, component='unit'))

A1 2BC area: : A
A1 2BC district: : 1
A1 2BC sector: : 2
A1 2BC unit: : BC
A12 3BC area: : A
A12 3BC district: : 12
A12 3BC sector: : 3
A12 3BC unit: : BC
AB1 2BC area: : AB
AB1 2BC district: : 1
AB1 2BC sector: : 2
AB1 2BC unit: : BC
AB12 3CD area: : AB
AB12 3CD district: : 12
AB12 3CD sector: : 3
AB12 3CD unit: : CD
A1B 2BC area: : A
A1B 2BC district: : 1B
A1B 2BC sector: : 2
A1B 2BC unit: : BC
AB1C 2DE area: : AB
AB1C 2DE district: : 1C
AB1C 2DE sector: : 2
AB1C 2DE unit: : DE


In [171]:
# create new columns with postcode components
df['pc_area'] = df['postcode'].apply(lambda x: get_postcode_component(x, 'area'))
df['pc_district'] = df['postcode'].apply(lambda x: get_postcode_component(x, 'district'))
df['pc_sector'] = df['postcode'].apply(lambda x: get_postcode_component(x, 'sector'))
df['pc_unit'] = df['postcode'].apply(lambda x: get_postcode_component(x, 'unit'))

In [172]:
df.nunique()

trans_id         302754
price             12043
date               7699
postcode          14066
type                  5
newbuild              2
tenure                3
paon               6122
saon               2321
street             8944
locality            119
town_city             1
district             11
county                4
ppd_type              2
record_status         1
pc_area               1
pc_district          25
pc_sector            10
pc_unit             387
dtype: int64

#### 2. Date

In [173]:
# convert dates to timestamp and discard meaningless hours/minutes data
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M'))

# sort by date
df = df.sort_values(by='date')

In [174]:
df.head()

,trans_id,price,date,postcode,type,newbuild,tenure,paon,saon,street,locality,town_city,district,county,ppd_type,record_status,pc_area,pc_district,pc_sector,pc_unit
153771,{9814661F-EF38-4E76-BC8B-CE526C9DA243},95950,1995-01-02,LS19 7WF,D,Y,F,8,,BALMORAL WAY,YEADON,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,19,7,WF
257025,{5DBCD3D4-07F0-4412-9379-DA6E6450B7B1},59950,1995-01-03,LS15 0PZ,S,Y,F,74,,FIELD END ROAD,LEEDS,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,15,0,PZ
561,{F09ADB38-2BD3-4013-BF09-7AE04A05F13E},187500,1995-01-03,LS17 7PX,D,N,F,40,,ALWOODLEY LANE,LEEDS,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,17,7,PX
137965,{203AC08D-673F-49E7-BBF1-9564E04D9089},59950,1995-01-03,LS27 8TS,S,Y,F,25,,MERLIN CLOSE,MORLEY,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,27,8,TS
104409,{8431EE39-650B-41F8-A0EC-A30B600809BD},32000,1995-01-03,LS14 6QH,S,N,F,59,,BROOKLANDS LANE,LEEDS,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,14,6,QH


## Price increases as a function of price

In [175]:
# filter by location and type
def filter_by(dataframe, **kwargs):
    
    """
    Filter a dataframe using arbitrary set of filters on column values 
    """
    
    for key, value in kwargs.items():
        dataframe = dataframe[dataframe[key] == value]
        
    return dataframe


In [188]:
# choose leeds
leeds = filter_by(df, pc_area='LS')

In [192]:
# filter by timescales spanning the beginning of Covid to present
leeds = leeds[(leeds['date'] > '2019-10-01')]

In [194]:
leeds

,trans_id,price,date,postcode,type,newbuild,tenure,paon,saon,street,locality,town_city,district,county,ppd_type,record_status,pc_area,pc_district,pc_sector,pc_unit,address
28197,{9B361207-255F-1904-E053-6B04A8C0EEB5},70500,2019-10-02,LS11 7DG,T,N,F,60,,HARLECH ROAD,,LEEDS,LEEDS,WEST YORKSHIRE,B,A,LS,11,7,DG,60LS11 7DG
28194,{965B6D92-63BC-95E4-E053-6C04A8C07729},59500,2019-10-02,LS8 5QG,T,N,F,41,,BAYSWATER CRESCENT,,LEEDS,LEEDS,WEST YORKSHIRE,B,A,LS,8,5,QG,41LS8 5QG
161869,{965B6D92-63C7-95E4-E053-6C04A8C07729},93500,2019-10-02,LS14 6LT,S,N,F,5,,HANSBY DRIVE,,LEEDS,LEEDS,WEST YORKSHIRE,B,A,LS,14,6,LT,5LS14 6LT
279497,{9FF0D96A-6184-11ED-E053-6C04A8C06383},186000,2019-10-02,LS11 9BJ,F,N,L,"VELOCITY SOUTH, 6",FLAT 25,CITY WALK,,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,11,9,BJ,"VELOCITY SOUTH, 6FLAT 25LS11 9BJ"
288599,{965B6D92-2D4B-95E4-E053-6C04A8C07729},92000,2019-10-02,LS11 7DB,S,N,F,22,,ATHA CRESCENT,BEESTON,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,11,7,DB,22LS11 7DB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106800,{D707E536-5AB9-0AD9-E053-6B04A8C067CC},212500,2022-01-11,LS7 4HA,S,N,F,81,,HAREHILLS LANE,,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,7,4,HA,81LS7 4HA
93163,{D707E536-5592-0AD9-E053-6B04A8C067CC},600000,2022-01-14,LS18 4HW,D,N,F,7A,,WOODSIDE HILL CLOSE,HORSFORTH,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,18,4,HW,7ALS18 4HW
199104,{D707E536-4A11-0AD9-E053-6B04A8C067CC},374500,2022-01-14,LS8 2AU,S,N,F,127,,KEDLESTON ROAD,,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,8,2,AU,127LS8 2AU
86008,{D707E536-50EE-0AD9-E053-6B04A8C067CC},302950,2022-01-17,LS5 3HG,T,N,F,54,,STATION PARADE,,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,5,3,HG,54LS5 3HG


In [ ]:
t = leeds['date'].max() -  leeds['date'].min()

In [228]:
from dateutil import relativedelta

def get_months_difference(timestamp1, timestamp2):
    
    r = relativedelta.relativedelta(timestamp1, timestamp2)

    months_diff = (r.years * 12) + r.months
    
    return months_diff

In [232]:
leeds['months_ago'] = leeds['date'].apply(lambda x: get_months_difference(datetime.today(), x))

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [231]:
datetime.today() - leeds['date'].max()

Timedelta('58 days 22:56:11.489750')

In [233]:
leeds

,trans_id,price,date,postcode,type,newbuild,tenure,paon,saon,street,locality,town_city,district,county,ppd_type,record_status,pc_area,pc_district,pc_sector,pc_unit,address,months_ago
28197,{9B361207-255F-1904-E053-6B04A8C0EEB5},70500,2019-10-02,LS11 7DG,T,N,F,60,,HARLECH ROAD,,LEEDS,LEEDS,WEST YORKSHIRE,B,A,LS,11,7,DG,60LS11 7DG,29
28194,{965B6D92-63BC-95E4-E053-6C04A8C07729},59500,2019-10-02,LS8 5QG,T,N,F,41,,BAYSWATER CRESCENT,,LEEDS,LEEDS,WEST YORKSHIRE,B,A,LS,8,5,QG,41LS8 5QG,29
161869,{965B6D92-63C7-95E4-E053-6C04A8C07729},93500,2019-10-02,LS14 6LT,S,N,F,5,,HANSBY DRIVE,,LEEDS,LEEDS,WEST YORKSHIRE,B,A,LS,14,6,LT,5LS14 6LT,29
279497,{9FF0D96A-6184-11ED-E053-6C04A8C06383},186000,2019-10-02,LS11 9BJ,F,N,L,"VELOCITY SOUTH, 6",FLAT 25,CITY WALK,,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,11,9,BJ,"VELOCITY SOUTH, 6FLAT 25LS11 9BJ",29
288599,{965B6D92-2D4B-95E4-E053-6C04A8C07729},92000,2019-10-02,LS11 7DB,S,N,F,22,,ATHA CRESCENT,BEESTON,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,11,7,DB,22LS11 7DB,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106800,{D707E536-5AB9-0AD9-E053-6B04A8C067CC},212500,2022-01-11,LS7 4HA,S,N,F,81,,HAREHILLS LANE,,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,7,4,HA,81LS7 4HA,2
93163,{D707E536-5592-0AD9-E053-6B04A8C067CC},600000,2022-01-14,LS18 4HW,D,N,F,7A,,WOODSIDE HILL CLOSE,HORSFORTH,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,18,4,HW,7ALS18 4HW,2
199104,{D707E536-4A11-0AD9-E053-6B04A8C067CC},374500,2022-01-14,LS8 2AU,S,N,F,127,,KEDLESTON ROAD,,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,8,2,AU,127LS8 2AU,2
86008,{D707E536-50EE-0AD9-E053-6B04A8C067CC},302950,2022-01-17,LS5 3HG,T,N,F,54,,STATION PARADE,,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,5,3,HG,54LS5 3HG,2


In [ ]:
def get_price_ratio()

In [182]:
#df.plot(x='date', y='price')
#plt.show()

In [179]:
# df = df.sort_values('date')
# 
# df['street'] = df['street'].astype(str)
# df['street'] = df['street'].apply(lambda x: x.lower())
# 
# df['days_ago'] = df['date'].apply(lambda x: datetime.today() - datetime.strptime(x, '%Y-%m-%d'))
# df['days_ago'] = df['days_ago'].apply(lambda x: str(x).split(' ')[0])
# df['days_ago'] = df['days_ago'].apply(lambda x: -int(x))
# df['months_ago'] = df['days_ago'].apply(lambda x: int(round( x/(365/12) ,0) ))

In [180]:
# def graph(postcode, housetype, streetname):
# 
#     import matplotlib.pyplot as plt
# 
#     postcode_type = df[  (df['postcode_area'] == f'{postcode}') & (df['type'] == f'{housetype}') ]
#     
#     # df for individual street
#     street = postcode_type[postcode_type['street'] == f'{streetname}']
#     
#     # price data for street
#     street_price = street.groupby(by=['months_ago']).agg({'price': 'median'})
#     
#     # moving average for street
#     street_price['MA'] = street_price['price'].rolling(window=12).mean()
#     
#     # price data for postcode area
#     postcode_type_price = postcode_type.groupby(by=['months_ago']).agg({'price': 'median'})
#     postcode_type_price = postcode_type_price.groupby(by=['months_ago']).agg({'price': 'median'})
#     
#     # moving average for postcode area
#     postcode_type_price['MA'] = postcode_type_price['price'].rolling(window=12).mean()
#     
#     plt.plot(street_price.index, street_price['price'].values,c='red',  label=f'{streetname}')
#     plt.plot(street_price.index, street_price['MA'].values, c='pink',  label=f'{streetname} moving average')
#     
#     plt.plot(postcode_type_price.index, postcode_type_price['MA'].values ,c='green',  label=f'{postcode, housetype} moving average')
#     plt.scatter(postcode_type_price.index, postcode_type_price['price'].values, c='blue', label=f'{postcode, housetype}')
#     
#     plt.rcParams["figure.figsize"]=(20,20)
#     plt.xticks(rotation=90)
#     plt.legend(loc='best')
#     plt.show()

In [181]:
def mortgage(principal, total_period, init_period, init_payment, init_rate, subs_rate, base_rate, over_payment):
    
    """
    Mortgage payment calculator. Returns dictionary of arrays
    
    principal: amount borrowed in arbitrary currency, e.g. 500000
    total_period: total mortgage duration in months
    init_period: initial term duration in months
    init_payment: initial monthly payment amount, e.g. 1205
    init_rate: initial interest rate expressed as a decimal. e.g. 0.0146 for 1.46%
    subs_rate: subsequent interest rate expressed as a decimal. e.g. 0.0376 for 3.76%
    base_rate: Bank of England base rate. e.g. 0.001 for 0.1%
    over_payment: optional fixed monthly overpayment
    """    
    
    # create arrays for plotting
    interest_values = []
    payment_values = []
    over_payment_values = []
    principal_values = []
    interest_cum_values = []
   
    # calculate initial max overpayment
    max_monthly_overpayment = principal*0.1*(1/12)
    
    # cumulative interest
    interest_cum = 0
    
    # payments for months of initial period
    for month in range(0, init_period):
        
        # update max available monthly overpayment each year
        if month % 12 == 0:
            max_monthly_overpayment = principal*0.1*(1/12)
            
        # calculate interest
        interest = principal*(1 + init_rate)**(1/12) - principal
        
        # add to interest for this month to total interest charged
        interest_cum += interest
    
        # add interest for current month
        principal += interest
     
        # subtract payment amount from principal
        if principal < init_payment:
            principal -= principal
            #stop payments if principal paid off
            init_payment=0 
            subs_payment=0
            over_payment=0
        else:
            principal -= init_payment       
            
        # subtract overpayment from principal
        if over_payment > max_monthly_overpayment:
            over_payment = max_monthly_overpayment

            if principal < over_payment:
                principal -= principal
                init_payment=0 
                subs_payment=0
                over_payment=0
            else:
                principal -= over_payment
        else:
            if principal < over_payment:
                principal -= principal
                init_payment=0 
                subs_payment=0
                over_payment=0
            else:
                principal -= over_payment
   
        # populate arrays for plotting
        interest_values.append(interest)
        interest_cum_values.append(interest_cum)
        payment_values.append(init_payment)
        over_payment_values.append(over_payment)
        principal_values.append(principal)
   
    # calculate payments for remaining balance at new rate
    subs_payment = ((((subs_rate+base_rate)/12))*((1+((subs_rate+base_rate)/12))**(total_period-init_period)) / (((1+((subs_rate+base_rate)/12))**(total_period-init_period)) - 1))*(principal)
    
    # payments for subsequent months
    for month in range(init_period, total_period):
        
        # interest added
        interest = principal*(1 + (subs_rate+base_rate))**(1/12) - principal
        
        # add to cumulative interest 
        interest_cum += interest
            
        # add interest for current month
        principal += interest
    
        # subtract (minimum) payment amount from principal
        if principal < subs_payment:
            principal -= principal
            #stop payments if principal paid off
            init_payment=0 
            subs_payment=0
            over_payment=0
        else:
            principal -= subs_payment
            
        # subtract overpayment amount from principal
        if principal < over_payment:
            principal -= principal
            #stop payments if principal paid off
            init_payment=0 
            subs_payment=0
            over_payment=0

        else:
            principal -= over_payment
        
        # populate arrays for plotting
        interest_values.append(interest)
        interest_cum_values.append(interest_cum)
        payment_values.append(subs_payment)
        over_payment_values.append(over_payment)
        principal_values.append(principal)
        
    results = dict.fromkeys(['Interest', 'Payments', 'Overpayments', 'Principal'])
    results['Interest'] = interest_values
    results['Payments'] = payment_values
    results['Overpayments'] = over_payment_values
    results['Principal'] = principal_values
    
    return results